# QM-Pipeline v7-minimal  
## Near-Zero Seam, LLR Clipping, and Comparative Ratio Preservation

### Scope and Non-Claims

This notebook does **not** propose, modify, or interpret any physical theory.  
It makes **no claims** about quantum dynamics, quantum states, measurement postulates, wavefunction collapse, or physical observables.

The object of study here is a **classical inference pipeline** operating on likelihood ratios under finite numerical precision and stabilisation constraints.

All quantities appearing in this notebook are **representational artefacts of inference**, not physical variables. Any resemblance to terminology used in quantum measurement theory reflects shared use of log-likelihood accumulation and thresholding, not a claim about underlying physics.

---

### Meaning of “Collapse” in This Notebook

The term *collapse* is used here **exclusively** to denote **loss of numerical distinguishability** caused by stabilisation procedures such as log-likelihood clipping, saturation, or thresholding.

It does **not** refer to physical wavefunction collapse, state reduction, decoherence, or any dynamical process.

In particular, the collapse demonstrated in this notebook occurs **despite** the underlying log-likelihood difference remaining finite and invariant. The loss of distinguishability is therefore a property of the numeric substrate and stabilisation policy, not of the inferred model.

---

### Role of Zero–Infinity Algebra (ZIA)

Zero–Infinity Algebra (ZIA) is used here **solely as a representational device** to preserve **comparative ratios** after standard real-valued arithmetic has saturated or collapsed.

ZIA is **not** treated as:
- a probability measure,
- a posterior,
- a physical quantity,
- a state,
- a dynamical variable,
- or an alternative arithmetic governing the underlying system.

All physical interpretation, if any, lies entirely upstream of this notebook.  
ZIA operates only on the outputs of an inference pipeline whose semantics are already fixed.

---

### What This Notebook Demonstrates

This notebook demonstrates **premature representational collapse**:  
a loss of comparative information induced by stabilisation procedures **before** any physical or statistical limit is reached.

The purpose is diagnostic, not interpretive: to isolate where and how information is destroyed by representation rather than by the underlying model.

***A lightweight Python reference implementation used for the examples in this paper is provided under a limited research and evaluation license; the canonical ZIA implementation will be released separately.***

In [ ]:
# Cell 1

import numpy as np
import math

print("np.finfo(float).tiny            =", np.finfo(float).tiny)
print("np.finfo(float).eps             =", np.finfo(float).eps)
print("np.nextafter(0.0, 1.0) (min sub) =", np.nextafter(0.0, 1.0))


In [ ]:
# Cell 2

m = 1.0
sigma2 = 16.0
N = 40

Delta = N * (1.0/8.0)  # = 5.0
R_star = math.exp(Delta)

eta = 1e-6
eps_list = [1e-50, 1e-100, 1e-200, 1e-300]

print("m      =", m)
print("sigma² =", sigma2)
print("N      =", N)
print("Δ      =", Delta)
print("R_star =", R_star)
print("η      =", eta)
print("eps_list =", eps_list)

assert abs(Delta - 5.0) < 1e-12
assert abs(R_star - 148.4131591025766) < 1e-12


In [ ]:
# Cell 3

def dllr(y):
    # Frozen per-step increment: δℓ(y) = y / 8
    return y / 8.0

print("dllr(1.0) =", dllr(1.0))
print("dllr(0.0) =", dllr(0.0))

assert abs(dllr(1.0) - 0.125) < 1e-15
assert dllr(0.0) == 0.0


In [ ]:
# Cell 4

y_post_S1 = [1.0] * N
y_post_S2 = [0.0] * N

print("Post-seam window length N =", N)
print("S1 first/last =", y_post_S1[0], y_post_S1[-1])
print("S2 first/last =", y_post_S2[0], y_post_S2[-1])


In [ ]:
# Cell 5

def llr_seam(eps, eta=eta):
    # ℓ_seam(ε) = log((1-η)/ε)
    return math.log((1.0 - eta) / eps)

print("Near-zero seam LLRs (must be large and finite):")
for eps in eps_list:
    ell = llr_seam(eps)
    print(f"  ε={eps:.0e}  ℓ_seam={ell:.6f}")
    assert math.isfinite(ell)


In [ ]:
# Cell 6

def llr_total(eps, y_post):
    return llr_seam(eps) + sum(dllr(y) for y in y_post)

print("Unclipped log-domain totals (ground truth comparative signal):")
for eps in eps_list:
    ell_S1 = llr_total(eps, y_post_S1)
    ell_S2 = llr_total(eps, y_post_S2)
    d = ell_S1 - ell_S2
    print(f"  ε={eps:.0e}  ℓ(S1)={ell_S1:.6f}  ℓ(S2)={ell_S2:.6f}  Δℓ={d:.12f}")
    assert abs(d - Delta) < 1e-12


In [ ]:
# Cell 7

LLR_CAP = 100.0  # frozen

def clip_llr(ell):
    return float(np.clip(ell, -LLR_CAP, +LLR_CAP))

def posterior_from_llr_clipped(ell_clip):
    # Equal priors; odds(A:B)=exp(ℓ). We use ℓ_clip as a stabilised proxy (demo only).
    e = math.exp(ell_clip)  # safe because ℓ_clip ∈ [-100, 100]
    pB = 1.0 / (1.0 + e)
    pA = 1.0 - pB
    return pA, pB

print("LLR clipping collapse demonstration (comparability destroyed):")
for eps in eps_list:
    ell_S1 = llr_total(eps, y_post_S1)
    ell_S2 = llr_total(eps, y_post_S2)

    ellc_S1 = clip_llr(ell_S1)
    ellc_S2 = clip_llr(ell_S2)

    pA1, pB1 = posterior_from_llr_clipped(ellc_S1)
    pA2, pB2 = posterior_from_llr_clipped(ellc_S2)

    print(f"  ε={eps:.0e}")
    print(f"    ℓ_clip(S1)={ellc_S1:.6f}  ℓ_clip(S2)={ellc_S2:.6f}  diff={ellc_S1-ellc_S2:.6f}")
    print(f"    posterior(S1): P(A)={pA1:.16f}  P(B)={pB1:.3e}")
    print(f"    posterior(S2): P(A)={pA2:.16f}  P(B)={pB2:.3e}")
    print(f"    posterior diffs: ΔP(A)={pA1-pA2:.16e}  ΔP(B)={pB1-pB2:.16e}")

    # Required collapse behaviour for this ε sweep:
    assert ellc_S1 == LLR_CAP and ellc_S2 == LLR_CAP
    assert (ellc_S1 - ellc_S2) == 0.0
    assert abs(pB1 - pB2) == 0.0


In [ ]:
# Cell 8 — Minimal ZIA model

# The provided `zia.py` in the same directory defines ZValue with the required fields (inf, res)
# and supports multiplication and division. In this notebook we use only that minimal subset.
from zia import ZValue

# Acceptance: pure-inf / pure-inf -> finite real ratio
x = ZValue.infinity(1.0)   # Z(1,0)
y = ZValue.infinity(2.0)   # Z(2,0)
ratio = x / y              # should be Z(0, 0.5)

print("x      =", x.to_nice_string())
print("y      =", y.to_nice_string())
print("x / y  =", ratio.to_nice_string())
print("float(x / y) =", float(ratio))

assert ratio.inf == 0.0 and abs(ratio.res - 0.5) < 1e-15


In [ ]:
# Cell 10 — ZIA mapping + ratio recovery
#
# Mapping used here (explicit):
# - Any finite LR factor r > 0 is mapped to Z(0, r) via ZValue.from_real(r).
# - The near-zero seam LR factor is r_seam = (1-η)/ε, which is finite for the frozen ε sweep,
#   so it is mapped the same way (no special-case algorithm changes; only representation differs).

def z_from_lr_factor(r):
    return ZValue.from_real(r)

def seam_lr_factor(eps, eta=eta):
    return (1.0 - eta) / eps

def z_total_for_stream(eps, y_post):
    Z = z_from_lr_factor(seam_lr_factor(eps))
    for y in y_post:
        # multiplicative update; same algorithm, different representation
        Z *= z_from_lr_factor(math.exp(dllr(y)))
    return Z

print("ZIA ratio recovery (should equal R_star for all ε):")
for eps in eps_list:
    Z_S1 = z_total_for_stream(eps, y_post_S1)
    Z_S2 = z_total_for_stream(eps, y_post_S2)

    R_Z = Z_S1 / Z_S2   # comparative ratio only
    R = float(R_Z)      # strict real expected here

    print(f"  ε={eps:.0e}  R_ZIA={R:.15f}  R_ZIA/R_star={R/R_star:.15f}")
    assert math.isfinite(R)
    assert abs(R - R_star) < 1e-9


v6 corresponds to the **ε = 0** limit (a structural zero at the seam). v7-minimal shows that the **same comparative structure already exists for ε > 0**, but standard stabilisation via **LLR clipping** destroys that information **before** the idealised limit is reached: S1 and S2 become indistinguishable even though the true Δℓ remains 5.0. ZIA provides a **continuous bridge** for the comparative ratio as ε → 0⁺, rather than a semantic jump.

In [ ]:
# ------------------------------------------------------------
# Cell 11 - Plot:
# Post-seam comparative window ratios
# Oracle vs ZIA
# Scalar log-domain explicitly unavailable
# ------------------------------------------------------------

import math
import matplotlib.pyplot as plt
from zia import ZValue

# --- configuration ---

window_sizes = list(range(1, N + 1))

# --- ORACLE: true post-seam window ratios ---

oracle_ratios = []
for w in window_sizes:
    delta = (
        sum(dllr(y) for y in y_post_S1[:w])
        - sum(dllr(y) for y in y_post_S2[:w])
    )
    oracle_ratios.append(math.exp(delta))

# --- ZIA: cumulative likelihood objects ---

Lambda_z_S1 = ZValue.infinity(1.0)
Lambda_z_S2 = ZValue.infinity(1.0)

zia_ratios = []

for y1, y2 in zip(y_post_S1, y_post_S2):
    Lambda_z_S1 = Lambda_z_S1 * ZValue.from_real(math.exp(dllr(y1)))
    Lambda_z_S2 = Lambda_z_S2 * ZValue.from_real(math.exp(dllr(y2)))

    r = Lambda_z_S1 / Lambda_z_S2
    assert r.is_real(), "ZIA ratio must be finite real post-seam"

    zia_ratios.append(r.as_real_strict())

# --- plotting ---

plt.figure(figsize=(8, 5))

plt.plot(
    window_sizes,
    oracle_ratios,
    linestyle="--",
    linewidth=2,
    label="Oracle (from raw record)",
)

plt.plot(
    window_sizes,
    zia_ratios,
    linestyle="-",
    linewidth=2,
    label="ZIA window ratio",
)

plt.axvline(
    x=0,
    color="red",
    linestyle="--",
    linewidth=1,
)


plt.xlabel("Post-seam window length")
plt.ylabel("Likelihood ratio")
plt.title("Post-seam comparative window ratios")

plt.text(
    0.08,
    0.2,
    "Scalar log-domain:\nwindow ratios undefined\n(inf − inf)",
    transform=plt.gca().transAxes,
    fontsize=11,
    color="gray",
    ha="left",
    va="center",
)

plt.legend()
plt.grid(True, linestyle=":")
plt.tight_layout()
plt.show()
